In [ ]:
import numpy as np
import pandas as pd

import cmath
from scipy.integrate import quad
from scipy.optimize import bisect
from scipy.stats import norm

In [ ]:
#### Black Scholes Function
def BlackScholesCore(CallPutFlag,DF,F,X,T,v):
    ## DF: discount factor
    ## F: Forward
    ## X: strike
    vsqrt=v*np.sqrt(T)
    d1 = (np.log(F/X)+(vsqrt*vsqrt/2.))/vsqrt
    d2 = d1-vsqrt
    if CallPutFlag:
        return DF*(F*norm.cdf(d1)-X*norm.cdf(d2))
    else:
        return DF*(X*norm.cdf(-d2)-F*norm.cdf(-d1))
    
##  Black-Scholes Pricing Function
def BlackScholes(CallPutFlag,S,X,T,r,d,v):
    ## r, d: continuous interest rate and dividend
    return BlackScholesCore(CallPutFlag,np.exp(-r*T),np.exp((r-d)*T)*S,X,T,v)

def Heston_P_Value(hestonParams,r,T,s0,K,typ):
    kappa, theta, sigma, rho, v0 = hestonParams
    return 0.5+(1./np.pi)*quad(lambda xi: Int_Function_1(xi,kappa,theta, sigma,rho,v0,r,T,s0,K,typ),0.,500.)[0]

def Int_Function_1(xi,kappa,theta,sigma,rho,v0,r,T,s0,K,typ):
    return (cmath.e**(-1j*xi*np.log(K))*Int_Function_2(xi,kappa,theta,sigma,rho,v0,r,T,s0,typ)/(1j*xi)).real

def Int_Function_2(xi,kappa,theta,sigma,rho,v0,r,T,s0,typ):
    if typ == 1:
        w = 1.
        b = kappa - rho*sigma
    else:
        w = -1.
        b = kappa
    ixi = 1j*xi
    d = cmath.sqrt((rho*sigma*ixi-b)*(rho*sigma*ixi-b) - sigma*sigma*(w*ixi-xi*xi))
    g = (b-rho*sigma*ixi-d) / (b-rho*sigma*ixi+d)
    ee = cmath.e**(-d*T)
    C = r*ixi*T + kappa*theta/(sigma*sigma)*((b-rho*sigma*ixi-d)*T - 2.*cmath.log((1.0-g*ee)/(1.-g)))
    D = ((b-rho*sigma*ixi-d)/(sigma*sigma))*(1.-ee)/(1.-g*ee)
    return cmath.e**(C + D*v0 + ixi*np.log(s0))

def heston_EuropeanCall(hestonParams,r,T,s0,K):
    a = s0*Heston_P_Value(hestonParams,r,T,s0,K,1)
    b = K*np.exp(-r*T)*Heston_P_Value(hestonParams,r,T,s0,K,2)
    return a-b

def heston_Impliedvol(hestonParams,r,T,s0,K):
    myPrice = heston_EuropeanCall(hestonParams,r,T,s0,K)
    ## Bisection algorithm when the Lee-Li algorithm breaks down
    def smileMin(vol, *args):
        K, s0, T, r, price = args
        return price - BlackScholes(True, s0, K, T, r, 0., vol)
    vMin = 0.000001
    vMax = 10.
    return bisect(smileMin, vMin, vMax, args=(K, s0, T, r, myPrice), rtol=1e-15, full_output=False, disp=True)

In [ ]:
#Price params
K= [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
T = [0.1, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.0]

#Model Params
n_samples = 5
v0_ = np.random.uniform(low = 0.0001, high=0.04, size=(n_samples))
rho_ = np.random.uniform(low = -0.95, high=-0.1, size=( n_samples))
sigma_ = np.random.uniform(low = 0.01, high= 1, size=(n_samples))
theta_ = np.random.uniform(low = 0.01, high= 0.2, size=(n_samples))
kappa_ = np.random.uniform(low = 1, high = 10 , size=(n_samples))

params = pd.DataFrame(np.array([v0_, rho_, sigma_, theta_, kappa_]).T, columns = ['v0', 'rho', 'sigma','theta','kappa'])
#Feller Condition 2kappa*theta > sigma^2
feller_index = 2*params['kappa']*params['theta'] > params['sigma']**2

In [ ]:
master_df = pd.DataFrame()
params_volsurface_map = {}
nSteps = 10
nPaths = int(1e3)

S0 = 1
r = 0
for loc, param in params[feller_index].iterrows():
    v0, rho, sigma, theta, kappa = param.values
    hestonParams = kappa, theta, sigma, rho, v0 
    vol_surface = pd.DataFrame()
    try:
        for strike in K:
            for maturity in T:
                iv = heston_Impliedvol(hestonParams,r,maturity,S0,strike)
                vol_surface = vol_surface.append(pd.DataFrame([[iv, strike, maturity]], columns = ['IV', 'Strike', 'Maturity']), ignore_index=True)

        params_volsurface_map[loc] = vol_surface
        print(f'Done: {loc}')        
        temp_ = vol_surface.set_index(['Strike', 'Maturity']).T
        temp_['v0'] = param.loc['v0']
        temp_['rho'] = param.loc['rho']
        temp_['sigma'] = param.loc['sigma']
        temp_['theta'] = param.loc['theta']
        temp_['kappa'] = param.loc['kappa']
        master_df = master_df.append(temp_, ignore_index=True)
        master_df.to_csv('data.csv', sep =";", decimal=',')
    except:
        #Some param gives error, hence the try,except.
        print(f'except: {loc}')
        pass